In [5]:
!pip install texthero
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, GRU
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import keras

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets  as tfds
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.utils import plot_model, to_categorical
from keras import backend as K
!pip install keras-lookahead

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
import texthero as hero
from texthero import preprocessing

In [7]:
cols = ['EventDescription', 'Address', 'IncidentCause', 'ActionTaken', 'FailedAssets', 'Locality','IncidentConsequence', 'CauseTechnical', 'CauseTechnical', 'Category']

In [8]:
df = pd.read_csv('/content/processed_dataset_IG.csv' , usecols=cols)
df.head()

,ActionTaken,Address,CauseTechnical,EventDescription,FailedAssets,IncidentCause,IncidentConsequence,Locality,Category
0,Crew isolated supply and undertook repairs,"PARA PARK, 1490 HENDY MAIN ROAD, PARAPARAP VIC...",Earth fault,A nearby customer reported sparking of electri...,Conductor (ABC),HVABC cable faulted midspan,Grassfire,Paraparap,OH Cable
1,Crew undertook repairs,"1154-1198 CHRISTIES ROAD, RAVENHALL VIC 3023",Leakage,A contractor reported that he had contacted an...,No failed asset,Contractor contacted earthing conductor,No Go Zone (contact),Ravenhall,Dug up
2,Crew isolated supply and undertook repairs,"326 DUNBARS ROAD, PETERBOROUGH VIC 3270",Corrosion,A field crew attending an outage found that a ...,Conductor (bare),Conductor broke due to rust,Serious risk to public safety (e.g. live condu...,Peterborough,Connection
3,Crew undertook repairs,"41-49 BANK STREET, SOUTH MELBOURNE VIC 3205",Leakage,Interfere and vandalism in substation. Unknown...,Indoor,Vandalism,Loss of supply;Damage to network assets,South Melbourne,Other
4,Crew isolated supply and undertook repairs,"199 WILSONS ROAD, WHITTINGTON VIC 3219",Leakage,A nearby customer reported that a high load ha...,Service conductor,Unknown high load contacted LV service cable,No Go Zone (contact);Damage to network assets,Whittington,Vehicle


In [9]:
import scipy 
from scipy import stats

le = LabelEncoder()
df['Category'] = le.fit_transform(df.Category.astype(str))
df['Category'].unique()

array([ 9,  5,  3, 10, 14,  6, 12, 11,  4,  2,  1,  0,  8, 13,  7])

In [10]:
def data(df):
    #new_df = df[['EventDescription', 'FailedAssets', 'IncidentCause', 'IncidentConsequence', 'IncidentType', 'Status', 'WeatherStation', 'Category']]
    #new_df.dropna(axis=0, inplace=True)
    features = df['EventDescription'] +' ' + df['Address'] + ' '+ df['IncidentCause'] +' ' + df['ActionTaken'] +\
    ' '+ df['FailedAssets'] +' ' + df['Locality']  + ' ' + df['IncidentConsequence'] + ' ' + \
    df['CauseTechnical'] + ' ' + df['CauseTechnical']
    target = df['Category']
    return features, target

features, target = data(df)
features

0       A nearby customer reported sparking of electri...
1       A contractor reported that he had contacted an...
2       A field crew attending an outage found that a ...
3       Interfere and vandalism in substation. Unknown...
4       A nearby customer reported that a high load ha...
                              ...                        
6438    A report came in to UE of a FMB sparking at 31...
6439    Report received from a resident to advise that...
6440    Report received of pole fire at incident locat...
6441    A customer called to report a tractor had hit ...
6442    concrete electrical cover outside of front of ...
Length: 6443, dtype: object

In [11]:
def preprocess_text():
    # cleaning steps
    cleaning_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_whitespace,
        preprocessing.remove_punctuation,
        preprocessing.remove_urls,
        preprocessing.remove_brackets,
        preprocessing.remove_stopwords,
        preprocessing.remove_digits,
        preprocessing.remove_angle_brackets,
        preprocessing.remove_curly_brackets,
        preprocessing.stem
    ]

    # apply pipeline to text
    clean_text = features.pipe(hero.clean, cleaning_pipeline)

    return clean_text


clean_text = preprocess_text()
clean_text

0       nearbi custom report spark electr line locat a...
1       contractor report contact earth cabl excav tre...
2       field crew attend outag found 22kv conductor b...
3       interfer vandal substat unknown third parti us...
4       nearbi custom report high load pull wire locat...
                              ...                        
6438    report came ue fmb spark st kilda st brighton ...
6439    report receiv resid advis crane made contact p...
6440    report receiv pole fire incid locat arriv crew...
6441    custom call report tractor hit wire brought gr...
6442    concret electr cover outsid front hous gave cl...
Length: 6443, dtype: object

In [12]:
x_train, x_test, y_train, y_test = train_test_split(clean_text, target, random_state=0, test_size=0.25, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4832,), (1611,), (4832,), (1611,))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [14]:
def tfidf():
    vectorizer = TfidfVectorizer(analyzer='word', max_features=1000)
    vectorizer.fit(clean_text)
    train_tfidf = vectorizer.transform(x_train)
    test_tfidf = vectorizer.transform(x_test)
    return train_tfidf, test_tfidf

train_tfidf, test_tfidf = tfidf()
train_tfidf.shape, test_tfidf.shape

((4832, 1000), (1611, 1000))

In [15]:
import numpy as np
oversample = SMOTE(random_state=0,n_jobs=-1,k_neighbors=5)
train_tfidf, y_train = oversample.fit_resample(train_tfidf, y_train)
print(f'Shape: {train_tfidf.shape}')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



Shape: (14820, 1000)


In [16]:
from xgboost import XGBClassifier

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, ConfusionMatrixDisplay

In [18]:
XGB=XGBClassifier()
XGB.fit(train_tfidf,y_train)
XGB_accuracy = accuracy_score(y_test,XGB.predict(test_tfidf))
print(XGB_accuracy.round(4)) 


0.8367


In [19]:

print(classification_report(y_test, XGB.predict(test_tfidf)))

              precision    recall  f1-score   support

           0       0.61      0.63      0.62        93
           1       0.92      0.81      0.86        67
           2       0.37      0.65      0.47        34
           3       0.78      0.84      0.81       210
           4       0.86      0.90      0.88       130
           5       0.92      0.89      0.91        92
           6       0.87      0.88      0.87       107
           7       0.00      0.00      0.00         8
           8       0.92      0.85      0.89        41
           9       0.75      0.63      0.68        60
          10       0.94      0.89      0.91       325
          11       0.66      0.57      0.61        83
          12       0.92      0.96      0.94       134
          13       0.50      0.67      0.57         6
          14       0.93      0.91      0.92       221

    accuracy                           0.84      1611
   macro avg       0.73      0.74      0.73      1611
weighted avg       0.84   